# Obtaining an appropriate execution Environment 
## ie docker-stacks/neuronunit-optimization:

Force use of development local repository. The repository must be the russell_dev branch of https://github.com/russelljjarvis/neuronunit
and the docker build is: `docker-stacks/neuronunit-optimization`, there are more graceful ways of achieving this.

If you lack `docker-stacks/neuronunit-optimization` try:

```
$git clone -b dev https://github.com/scidash/docker-stacks.git
$cd docker-stacks
$sudo bash build-all```

The best way will be to integrate this doc chapter5 and its supporting code 
from https://github.com/russelljjarvis/neuronunit -> https://github.com/scidash/neuronunit

# Invocation of the jupyter note book:

On my local machine the development repo of neuronunit is located here:

```
$HOME/git
# I navigate to that directory and execute:

$docker run -p 8888:8888 -v \
`pwd`:/home/jovyan/mnt scidash/neuronunit-optimization \
jupyter notebook --ip=0.0.0.0 --NotebookApp.token=\"\" \ 
--NotebookApp.disable_check_xsrf=True 
```

In [1]:
import sys
import os
THIS_DIR = os.path.dirname(os.path.realpath('chapter5.ipynb'))
this_nu = os.path.join(THIS_DIR,'../')
sys.path.insert(0,this_nu)
import neuronunit


In [ ]:
from neuronunit.tests import get_neab
from neuronunit.tests import utilities as outils                                    
from neuronunit.tests import model_parameters as modelp
import numpy as np
model = outils.model          
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 3
        gap.NGEN = 3
        # The population of genes is 12
        gap.MU = 12 
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]
        # call the actual Genetic Algorithm with Optimization parameters: number of generation (NGEN = 3)
        # and gene population size (MU = 12)
        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        attributes = [ i.attrs for i in vmpop ]
        rheobases = [ i.rheobase for i in vmpop ]
        scores = [ i.score for i in vmpop ]        
        parameters = [ i.attrs for i in vmpop ]
        
        data_tuples = (vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        return pop, data_tuples
        
    def _get_optimization_parameters(self, data_tuples):
        # Your specific unpacking of tuples that _optimize returns
        # vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        _ , parameters , scores, _ , _ , _ , _ = zip(*data_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        # The Class users version of optimize
        # where details are hidden in _optimizae
        
        # Inputs: 
        # a Izhihikitch model specified in NML, but implemented with a NEURONbackend type
        # from neuronunit. Modelp a formatated dictionary of model parameters
        # where keys are Izhikitich parameters, and values are parameter ranges.
        # Outputs:
        # the optimal model, the scores as pandas data frame.
        # data_tuples: other data about models from the converged gene population
        # like resulting rheobase values from the converged genes, pandas score arrays
        # the genes corresponding to attributes of the pareto front (in a raw format).
        '''

        # Do optimization including repeated calls to judge
        models, data_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(data_tuples)
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.
        # scores is a list of pandas dataframes for the converged gene population.
        # It might be good to convert it into one big panda table if I knew how.
        return model, scores, data_tuples

    
t = Test()
model,scores,data_tuples = t.optimize(model,modelp)   

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


0 39.5309312255
1 9.01815779729e-05
2 0.00132055823175
3 -55.3884950323
4 -50.9395557665
5 -37.1403940048
6 0.761692292387
7 -63.0570348085
8 -4.40690996359e-09
9 0.16229074678
0 38.8911531008
1 9.99910284218e-05
2 0.00117985306869
3 -59.193324137
4 -70.2305584324
5 -34.5100679542
6 0.788385748028
7 -48.3813431566
8 -4.0545419299e-09
9 0.165739235014
0 39.8294164422
1 9.3620140521e-05
2 0.00136098129646
3 -59.7582515777
4 -69.4379967768
5 -49.0843529553
6 0.899558767906
7 -71.5312516256
8 -4.13252796468e-09
9 0.0752294652314
0 37.5155325549
1 0.000100048521614
2 0.000665076217596
3 -56.2454268648
4 -56.2797829552
5 -30.9577429262
6 0.200569985787
7 -48.807898246
8 -3.65147209721e-09
9 0.107934559161
0 35.3638290512
1 0.000106522354481
2 0.000640660224148
3 -55.044672003
4 -61.3150359058
5 -45.1636657056
6 0.884491355241
7 -45.3516770956
8 -3.52031891333e-09
9 0.0988677009072
0 32.4914031119
1 0.000104827153232
2 0.000776614242467
3 -57.8240832299
4 -56.1321246369
5 -40.0207599887
6 0.6

<class 'float'>
376.54320987654324 vanilla {'injected_square_current': {'delay': '100.0', 'amplitude': '62.96296296296297', 'duration': '1000.0'}} (0, <neuronunit.tests.utilities.VirtualModel object at 0x7fc6f106ef28>, 376.54320987654324)
{}
{}
{}


<class 'float'>
141.0493827160494 vanilla {'injected_square_current': {'delay': '100.0', 'amplitude': '376.54320987654324', 'duration': '1000.0'}} (0, <neuronunit.tests.utilities.VirtualModel object at 0x7fc6f1182828>, 141.0493827160494)
{}
{}
{}


<class 'float'>
106.94444444444446 vanilla {'injected_square_current': {'delay': '100.0', 'amplitude': '141.0493827160494', 'duration': '1000.0'}} (0, <neuronunit.tests.utilities.VirtualModel object at 0x7fc6f1182588>, 106.94444444444446)
{}
{}
{}


...

<class 'float'>
188.19444444444446 vanilla {'injected_square_current': {'delay': '100.0', 'amplitude': '364.8148148148148', 'duration': '1000.0'}} (1, <neuronunit.tests.utilities.VirtualModel object at 0x7fc6ec3c8160>, 188.19444444444446)
{}
{}
{}


<class 'float'>
124.53703703703704 vanilla {'injected_square_current': {'delay': '100.0', 'amplitude': '188.19444444444446', 'duration': '1000.0'}} (1, <neuronunit.tests.utilities.VirtualModel object at 0x7fc6ec3ce940>, 124.53703703703704)
{}
{}
{}


<class 'float'>
320.1388888888889 vanilla {'injected_square_current': {'delay': '100.0', 'amplitude': '124.53703703703704', 'duration': '1000.0'}} (1, <neuronunit.tests.utilities.VirtualModel object at 0x7fc6ec3cea90>, 320.1388888888889)
{}
{}
{}


0 [39.530931225513612, 9.018157797291147e-05, 0.0013205582317519274, -55.38849503225245, -50.939555766476175, -37.140394004809465, 0.76169229238668967, -63.057034808519589, -4.4069099635860598e-09, 0.16229074677969846]
<class 'int'> <class 'deap.creator.Individual'>
1 [37.130797747855958, 0.00010209816184074588, 0.0012040827834609543, -55.258716175536875, -60.610041147310532, -37.42061718098288, 0.91161758597945319, -66.252568327167666, -4.2471008087963327e-09, 0.09709888074260753]
<class 'int'> <class 'deap.creator.Individual'>
2 [38.891153100770879, 9.9991028421773844e-05, 0.0011798530686923216, -59.193324136978383, -70.230558432360382, -34.510067954168761, 0.78838574802763606, -48.381343156584137, -4.0545419299021334e-09, 0.1657392350140405]
<class 'int'> <class 'deap.creator.Individual'>
3 [39.829416442206679, 9.3620140520966379e-05, 0.0013609812964611736, -59.758251577665895, -69.437996776817386, -49.084352955347128, 0.89955876790583922, -71.531251625576971, -4.1325279646760665e-0

{}
{}
{}
False
{}
<class 'float'>
current 77.00617283950618 spikes 1
{}
<class 'float'>
current 77.16049382716051 spikes 1
{}
<class 'float'>
current 77.31481481481482 spikes 1
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
<class 'float'>
current 382.8703703703704 spikes 1
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
<class 'float'>
current 174.88425925925927 spikes 1
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
<class 'float'>
current 384.7222222222223 spikes 1
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 140.79218106995887 spikes 1
{}
<class 'float'>
current 140.84362139917698 spikes 1
{}
<class 'float'>
current 140.8950617283951 spikes 1
{}
<class 'float'>
{}
{}

In [ ]:
modelp
import pandas as pd
models = data_tuples[1]
sc = pd.DataFrame(scores[0])
for j,i in enumerate(models):
        i.name = attributes[j]
sc

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0

In [ ]:
rheobases=data_tuples[5][0]

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0


In [ ]:
rheobases[0]


In [ ]:

sc1 = pd.DataFrame(scores[1])
sc1


In [ ]:
rheobases[1]


In [ ]:
data=[ models[1].name ]
model_values1 = pd.DataFrame(data)        
model_values1

In [ ]:
models[1].name        

The code below is used to get the differences between values obtained via brute force, and those obtained otherwise.
It displays the differences in parameter values as pandas data tables.
    
I have knowingly violated Github conventions by adding data (a pickled file, as well as sources to the repository). 
The justification being that ground_error (the ground truth to compare against Genetic Algorithm outputs). 
Takes a prohibitively long time to generate, and therefore detracts from notebooking philosophy.

In [ ]:
import pickle
import pandas as pd

try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    # The exception code is only skeletal, it would not actually work, but its the right principles.
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))

# ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
# pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = data_tuples[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])




In [ ]:
#These are the differences in attributes found via brute force versus the genetic algorithm. For the top two candidates.

df_0

In [ ]:
df_1
